## Муравьиный алгоритм

В этой части я реализую муравьиный алгоритм — метод оптимизации, вдохновлённый поведением реальных муравьёв при поиске кратчайшего пути к источнику пищи. Алгоритм особенно хорошо подходит для задач коммивояжёра, поиска маршрутов и других комбинаторных задач.

В качестве практической задачи рассматривается планирование оптимального маршрута по парку развлечений Диснейленд, чтобы минимизировать общее время ожидания в очередях. Для этого я использую актуальные данные с сайта queue-times.com, который предоставляет информацию о текущем времени ожидания на аттракционы.

Каждый «муравей» в алгоритме будет моделировать посетителя парка, пытающегося найти лучший маршрут по аттракционам с учётом текущей загруженности. Муравьи будут оставлять виртуальные феромоны на маршрутах, которые приводят к более короткому времени ожидания, тем самым усиливая привлекательность удачных путей для последующих агентов.

Таким образом, цель алгоритма — найти наиболее выгодную последовательность посещения аттракционов, которая минимизирует время в очередях и делает пребывание в парке максимально эффективным и комфортным.

In [42]:
%use kandy
%use ktor-client

### Подготовка функций и данных о парке

In [43]:
@Serializable
data class Park(
  val id: Int,
  val name: String,
  val country: String,
  val continent: String,
  val latitude: Double,
  val longitude: Double,
  val timezone: String
)

@Serializable
data class Company(
  val id: Int,
  val name: String,
  val parks: List<Park>
)

In [47]:
fun fetchParksData(): List<Company> =
  Json.decodeFromString(http.get("https://queue-times.com/parks.json").body())

fetchParksData()
  .find { it.name.contains("Disney") }
  ?.parks
  ?.map { it.name }

[Animal Kingdom, Disney California Adventure, Disney Hollywood Studios, Disney Magic Kingdom, Disneyland, Disneyland Hong Kong, Disneyland Park Paris, Epcot, Shanghai Disney Resort, Tokyo Disneyland, Tokyo DisneySea, Walt Disney Studios Paris]

In [50]:
fetchParksData()
  .find { it.name.contains("Disney") }
  ?.parks
  ?.find { it.name.contains("Shanghai") }

[Park(id=30, name=Shanghai Disney Resort, country=China, continent=Asia, latitude=31.144, longitude=121.657, timezone=Asia/Shanghai)]

In [63]:
@Serializable
data class Ride(
  val id: Int,
  val name: String,
  val is_open: Boolean,
  val wait_time: Int,
  val last_updated: String,
)

@Serializable
data class Land(
  val id: Int,
  val name: String,
  val rides: List<Ride>
)

@Serializable
data class QueuesInfo(
  val lands: List<Land>,
  val rides: List<Ride>,
)

In [68]:
fun fetchParkQueueTimesData(parkId: Int): QueuesInfo =
  Json.decodeFromString(http.get("https://queue-times.com/parks/$parkId/queue_times.json").body())

fetchParksData()
  .find { it.name.contains("Disney") }
  ?.parks
  ?.find { it.name.contains("California") }
  ?.run { fetchParkQueueTimesData(parkId = id) }
  ?.lands
  ?.map { it.rides }
  ?.flatten()
  ?.map { "${it.name}: ожидание ${it.wait_time} мин."}

[Guardians of the Galaxy - Mission: BREAKOUT!: ожидание 35 мин., WEB SLINGERS: A Spider-Man Adventure: ожидание 30 мин., WEB SLINGERS: A Spider-Man Adventure Single Rider: ожидание 0 мин., Luigi's Rollickin' Roadsters: ожидание 15 мин., Mater's Junkyard Jamboree: ожидание 5 мин., Radiator Springs Racers: ожидание 65 мин., Radiator Springs Racers Single Rider: ожидание 0 мин., Grizzly River Run: ожидание 5 мин., Redwood Creek Challenge Trail: ожидание 0 мин., Soarin' Around the World: ожидание 30 мин., Animation Academy: ожидание 0 мин., Mickey's PhilharMagic: ожидание 0 мин., Monsters, Inc. Mike & Sulley to the Rescue!: ожидание 15 мин., Sorcerer's Workshop: ожидание 0 мин., Turtle Talk with Crush: ожидание 0 мин., Golden Zephyr: ожидание 5 мин., Goofy's Sky School: ожидание 0 мин., Silly Symphony Swings: ожидание 10 мин., Silly Symphony Swings Single Rider: ожидание 0 мин., The Little Mermaid - Ariel's Undersea Adventure: ожидание 5 мин., Games of Pixar Pier: ожидание 5 мин., Incredic

In [69]:
data class ExtendedRide(
  val ride: Ride,
  val x: Double,
  val y: Double,
)

In [72]:
fun generateRides(
  companyName: String,
  parkName: String
): List<ExtendedRide> {
  val rides = fetchParksData()
    .find { it.name.contains(companyName) }
    ?.parks
    ?.find { it.name.contains(parkName) }
    ?.run { fetchParkQueueTimesData(parkId = id) }
    ?.lands
    ?.map { it.rides }
    ?.flatten()
    ?.filter { it.is_open }
    .orEmpty()

  val count = rides.count()

  return rides.mapIndexed { index, ride ->
    ExtendedRide(
      ride = ride,
      x = sin(index * 1.0 / count * PI * 2),
      y = cos(index * 1.0 / count * PI * 2),
    )
  }
}

In [76]:
val rides = generateRides(companyName = "Disney", parkName = "California")

plot {
  points {
    x(rides.map { it.x })
    y(rides.map { it.y })
  }

  layout {
    title = "Координаты аттракционов"
  }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Wd599C"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Координаты аттракционов"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.0,0.21497044021102407,0.4198891015602646,0.6051742151937651,0.7621620551276365,0.8835120444460229,0.9635499925192229,0.9985334138511238,0.9868265225415261,0.9289767198167914,0.8276889981568906,0.6876994588534235,0.5155538571770221,0.3193015301359802,0.10811901842394192,-0.10811901842394168,-0.31930153013597995,-0.5155538571770214,-0.6876994588534234,-0.8276889981568905,-0.9289767198167914,-0.9868265225415261,-0.9985334138511239,-0.9635499925192229,-0.8835120444460233,-0.7621620551276368,-0.6051742151937656,-0.41988910156026493,-0.21497044021102438],
"y":[1.0,0.9766205557100867,0.907575419670957,0.7960930657056438,0.6473862847818277,0.46840844069979015,0.267528338529221,0.05413890858541761,-0.16178199655276473,-0.37013815533991445,-0.5611870653623823,-0.7259954919231306,-0.856857176167589,-0.9476531711828023,-0.9941379571543596,-0.9941379571543596,-0.9476531711828025,-0.8568571761675894,-0.7259954919231307,-0.5611870653623825,-0.37013815533991445,-0.16178199655276476,0.0541389085854167,0.267528338529221,0.46840844069978954,0.6473862847818272,0.7960930657056434,0.9075754196709569,0.9766205557100867]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"point",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"float",
"column":"y"
}]
}
}],
"spec_id":"14"
};
 var containerDiv = document.getElementById("Wd599C");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 
 
 
 
 -0.5 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 Координаты аттракционов 
 
 
 
 
 y 
 
 
 
 
 x

In [78]:
fun getDistanceBetweenRides(
  rideA: ExtendedRide,
  rideB: ExtendedRide
) = Math.sqrt(rideA.x * rideB.x + rideA.y * rideB.y)

### Муравей

In [ ]:
class Ant(val rides: List<ExtendedRide>) {

  val visitedAttractions = rides.shuffled().toMutableList()

  fun visitAttraction() {}

  fun visitRandomAttraction() {

  }

  fun visitProbabilisticAttraction() {}

  fun rouletteWheelSelection() {}

  fun getDistanceTraveled() = visitedAttractions
    .takeIf { it.size > 1 }
    ?.zipWithNext()
    ?.sumOf { (from, to) -> getDistanceBetweenRides(from, to) }
    ?: 0.0
}